In [53]:
from pyspark.sql import SparkSession
from pyspark import SparkContext


In [54]:
sc= SparkContext()
sqlContext = SparkSession(sc)

In [55]:
data = sqlContext.read.csv("/home/smauggy/Downloads/500_Cities__Local_Data_for_Better_Health.csv",header='true')

In [56]:
data.printSchema()

root
 |-- Year: string (nullable = true)
 |-- StateAbbr: string (nullable = true)
 |-- StateDesc: string (nullable = true)
 |-- CityName: string (nullable = true)
 |-- GeographicLevel: string (nullable = true)
 |-- DataSource: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- UniqueID: string (nullable = true)
 |-- Measure: string (nullable = true)
 |-- Data_Value_Unit: string (nullable = true)
 |-- DataValueTypeID: string (nullable = true)
 |-- Data_Value_Type: string (nullable = true)
 |-- Data_Value: string (nullable = true)
 |-- Low_Confidence_Limit: string (nullable = true)
 |-- High_Confidence_Limit: string (nullable = true)
 |-- Data_Value_Footnote_Symbol: string (nullable = true)
 |-- Data_Value_Footnote: string (nullable = true)
 |-- PopulationCount: string (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- CategoryID: string (nullable = true)
 |-- MeasureId: string (nullable = true)
 |-- CityFIPS: string (nullable = true)
 |-- TractFIPS: str

In [57]:
data.take(2)

[Row(Year=u'2014', StateAbbr=u'US', StateDesc=u'United States', CityName=None, GeographicLevel=u'US', DataSource=u'BRFSS', Category=u'Prevention', UniqueID=u'59', Measure=u'Current lack of health insurance among adults aged 18\u201364 Years', Data_Value_Unit=u'%', DataValueTypeID=u'AgeAdjPrv', Data_Value_Type=u'Age-adjusted prevalence', Data_Value=u'14.9', Low_Confidence_Limit=u'14.6', High_Confidence_Limit=u'15.2', Data_Value_Footnote_Symbol=None, Data_Value_Footnote=None, PopulationCount=u'308745538', GeoLocation=None, CategoryID=u'PREVENT', MeasureId=u'ACCESS2', CityFIPS=None, TractFIPS=None, Short_Question_Text=u'Health Insurance'),
 Row(Year=u'2014', StateAbbr=u'US', StateDesc=u'United States', CityName=None, GeographicLevel=u'US', DataSource=u'BRFSS', Category=u'Prevention', UniqueID=u'59', Measure=u'Current lack of health insurance among adults aged 18\u201364 Years', Data_Value_Unit=u'%', DataValueTypeID=u'CrdPrv', Data_Value_Type=u'Crude prevalence', Data_Value=u'14.1', Low_Co

In [92]:
print data.count()

810103


In [147]:
from pyspark.mllib.stat import Statistics

from pyspark.sql import functions as F

data.agg(F.min(data.Year),F.max(data.Year), F.avg(data.PopulationCount), F.approx_count_distinct(data.CityName)).collect()


cities = data.groupBy(data.CityName)

data.agg(F.approxCountDistinct(data.Category)).collect()


In [150]:

top_30 = cities.agg(F.avg(data.Data_Value)).rdd.takeOrdered(30, key = lambda x: -x[1])
lowest_30 = cities.agg(F.avg(data.Data_Value)).rdd.takeOrdered(30, key = lambda x: x[1])

In [61]:
import pandas as pd

top_cities_30 = pd.DataFrame(top_30, columns=['city','percentage'])
least_cities_30 = pd.DataFrame(lowest_30, columns=['city','percentage'])


In [62]:
from bokeh.charts import Bar, show
from bokeh.charts.attributes import ColorAttr, CatAttr
from bokeh.models import PrintfTickFormatter
from bokeh.io import output_notebook
output_notebook()
p = Bar(top_cities_30, values='percentage', title="Top 30 cities in US with lack of insurance % ", label=CatAttr(columns=['city'], sort=False),legend=False)
p.yaxis.formatter=PrintfTickFormatter(format='%0.00f %%')
show(p, notebook_handle=True)

p1 = Bar(least_cities_30, values='percentage', title="Bottom 30 cities in US with lack of insurance % ", label=CatAttr(columns=['city'], sort=False),legend=False)
p1.yaxis.formatter=PrintfTickFormatter(format='%0.0f %%')
show(p1, notebook_handle=True)


Loading BokehJS ...

In [64]:

from bs4 import BeautifulSoup
import urllib2
 
wiki = "https://en.wikipedia.org/wiki/List_of_highest-income_metropolitan_statistical_areas_in_the_United_States"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

Rank =""
City=""
Population =""
Percapita =""

table = soup.find("table", { "class" : "toccolours sortable" })

f = open('output.csv', 'w')

for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 4:
        Rank = cells[0].findAll(text=True)
        if 'href' in getattr(cells[1], 'attrs', {}):
            City = cells[1].findAll('a')
        else:
            City = cells[1].findAll(text=True)
        Population = cells[2].findAll(text=True)
        Percapita = cells[3].findAll(text=True)
        write_to_file = str(Rank) + "\t" + str(City) + "\t" + str(Population) + "\t" + str(Percapita) + "\n"
        print write_to_file
        f.write(write_to_file)
f.close()


[u'1']	[u'San Francisco\u2013Oakland\u2013San Jose, California', u' CMSA']	[u'7,239,362']	[u'$63,024']

[u'2']	[u'New York\u2013Northern New Jersey\u2013Long Island, New York\u2013New Jersey\u2013Connecticut\u2013Pennsylvania', u' CMSA']	[u'21,199,865']	[u'$59,799']

[u'3']	[u'Washington\u2013Baltimore, District of Columbia\u2013Maryland\u2013Virginia\u2013West Virginia', u' CMSA']	[u'7,608,070']	[u'$57,291']

[u'4']	[u'Anchorage, Alaska', u' MSA']	[u'260,283']	[u'$56,787']

[u'5']	[u'Minneapolis\u2013St. Paul, Minnesota\u2013Wisconsin', u' CMSA']	[u'3,615,902']	[u'$54,304']

[u'6']	[u'Boston\u2013Worcester\u2013Lawrence, Massachusetts\u2013New Hampshire\u2013Maine\u2013Connecticut', u' CMSA']	[u'5,819,101']	[u'$52,792']

[u'7']	[u'Hartford, Connecticut', u' MSA']	[u'1,183,110']	[u'$52,188']

[u'8']	[u'Atlanta, Georgia', u' MSA']	[u'4,112,198']	[u'$51,948']

[u'9']	[u'Honolulu, Hawaii', u' MSA']	[u'876,156']	[u'$51,914']

[u'10']	[u'Rochester, Minnesota', u' MSA']	[u'124,277']	[u'$51,3

In [65]:

Cities_Income = pd.read_csv("output.csv",sep="\t",names=['rank','city','population','median_income'])